##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 前処理レイヤーを使用する

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/keras/preprocessing_layers"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で実行</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/guide/keras/preprocessing_layers.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a> </td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/guide/keras/preprocessing_layers.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     GitHubでソースを表示</a></td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/guide/keras/preprocessing_layers.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a> </td>
</table>

## Keras 前処理レイヤー

Keras 前処理レイヤー API を使用すると、開発者は Keras ネイティブの入力処理パイプラインを構築できます。これらの入力処理パイプラインは、Keras 以外のワークフローで独立した前処理コードとして使用し、Keras モデルと直接組み合わせて、Keras SavedModel の一部としてエクスポートできます。

Keras 前処理レイヤーを使用すると、真にエンドツーエンドのモデル（生の画像または生の構造化データを入力として受け入れるモデルや特徴の正規化または特徴値のインデックス作成を独自に処理するモデル）を構築およびエクスポートできます。

## 利用可能な前処理レイヤー

### 主な前処理レイヤー

- `TextVectorization` レイヤー: 生の文字列を、`Embedded` レイヤーまたは `Dense` レイヤーで読み取ることができるエンコードされた表現に変換します。
- `Normalization` レイヤー: 入力する特徴の特徴ごとの正規化を実行します。

### 構造化データの前処理レイヤー

これらのレイヤーは、構造化データのエンコードと特徴量エンジニアリング用です。

- `CategoryEncoding` レイヤー: 整数のカテゴリ特徴量をワンホット、マルチホット、または TF-IDF の密な表現に変換します。
- `Hashing` レイヤー: 「ハッシュトリック」とも呼ばれるカテゴリ特徴量ハッシングを実行します。
- `Discretization` レイヤー: 連続する数値特徴量を整数のカテゴリ特徴量に変換します。
- `StringLookup` レイヤー: 文字列カテゴリ値を `Embedding` レイヤーまたは `Dense` レイヤーで読み取れる埋め込み表現に変換します。
- `IntegerLookup` レイヤー: 整数のカテゴリ値を `Embedding` レイヤーまたは `Dense` レイヤーで読み取れる埋め込み表現に変換します。
- `CategoryCrossing`レイヤー：カテゴリ特徴量を共起特徴量に結合します。 たとえば、 特徴量値「a」と「b」がある場合、「a と b が同時に存在する」という組み合わせの特徴量を提供できます。

### 画像前処理レイヤー

これらのレイヤーは、画像モデルの入力を標準化するためのものです。

- `Resizing` レイヤー: 画像のバッチサイズをターゲットサイズに変更します。
- `Rescaling` レイヤー: 画像のバッチの値をスケーリングし直してからオフセットします（たとえば、`[0, 255]` の範囲の入力から `[0, 1]` の範囲の入力に移動します。
- `CenterCrop` レイヤー: 画像のバッチの中央のトリミングを返します。

### 画像データ拡張レイヤー

以下のレイヤーは、ランダムな拡張変換を画像のバッチに適用します。これらはトレーニング中にのみアクティブになります。

- `RandomCrop` レイヤー
- `RandomFlip` レイヤー
- `RandomTranslation` レイヤー
- `RandomRotation` レイヤー
- `RandomZoom` レイヤー
- `RandomHeight` レイヤー
- `RandomWidth` レイヤー

## `adapt()` メソッド

一部の前処理レイヤーには、トレーニングデータのサンプルに基づいて計算する必要がある内部状態があります。ステートフル前処理レイヤーのリストは次のとおりです。

- `TextVectorization`: 文字列トークンと整数インデックス間のマッピングを保持します。
- `StringLookup`と`IntegerLookup`: 入力値と整数インデックスの間のマッピングを保持します。
- `Normalization`: 特徴量の平均と標準偏差を保持します。
- `Discretization`: 値バケットの境界に関する情報を保持します。

重要な点は、これらのレイヤーは**トレーニング対象外**であることです。状態はトレーニング中に設定されません。そのため、**トレーニングの前**に、「適応」と呼ばれるステップで設定する必要があります。

`adapt()`メソッドを使用して、トレーニングデータに前処理レイヤーを公開することにより、前処理レイヤーの状態を設定します。

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

data = np.array([[0.1, 0.2, 0.3], [0.8, 0.9, 1.0], [1.5, 1.6, 1.7],])
layer = preprocessing.Normalization()
layer.adapt(data)
normalized_data = layer(data)

print("Features mean: %.2f" % (normalized_data.numpy().mean()))
print("Features std: %.2f" % (normalized_data.numpy().std()))

`adapt()`メソッドは、Numpy 配列または`tf.data.Dataset`オブジェクトのいずれかを取ります。`StringLookup`および`TextVectorization`の場合、文字列のリストを渡すこともできます。

In [ ]:
data = [
    "ξεῖν᾽, ἦ τοι μὲν ὄνειροι ἀμήχανοι ἀκριτόμυθοι",
    "γίγνοντ᾽, οὐδέ τι πάντα τελείεται ἀνθρώποισι.",
    "δοιαὶ γάρ τε πύλαι ἀμενηνῶν εἰσὶν ὀνείρων:",
    "αἱ μὲν γὰρ κεράεσσι τετεύχαται, αἱ δ᾽ ἐλέφαντι:",
    "τῶν οἳ μέν κ᾽ ἔλθωσι διὰ πριστοῦ ἐλέφαντος,",
    "οἵ ῥ᾽ ἐλεφαίρονται, ἔπε᾽ ἀκράαντα φέροντες:",
    "οἱ δὲ διὰ ξεστῶν κεράων ἔλθωσι θύραζε,",
    "οἵ ῥ᾽ ἔτυμα κραίνουσι, βροτῶν ὅτε κέν τις ἴδηται.",
]
layer = preprocessing.TextVectorization()
layer.adapt(data)
vectorized_text = layer(data)
print(vectorized_text)

さらに、適応可能なレイヤーは、コンストラクタ引数または重みの割り当てを介して状態を直接設定するオプションを常に公開します。意図した状態値がレイヤー構築時にわかっている場合、または`adapt()`呼び出しの外で計算される場合は、レイヤーの内部計算に依存せずに設定できます。例えば、`TextVectorization`、`StringLookup`、または、`IntegerLookup`レイヤーの外部語彙ファイルがすでに存在する場合、レイヤーのコンストラクタ引数で語彙ファイルへのパスを渡すことにより、それらをルックアップテーブルに直接読み込めます。

以下の例では、事前に計算された語彙を使用して`StringLookup`レイヤーをインスタンス化します。

In [ ]:
vocab = ["a", "b", "c", "d"]
data = tf.constant([["a", "c", "d"], ["d", "z", "b"]])
layer = preprocessing.StringLookup(vocabulary=vocab)
vectorized_data = layer(data)
print(vectorized_data)

## モデルの前またはモデル内のデータの前処理

前処理レイヤーを使用する方法は 2 つあります。

**オプション 1:** 次のように、それらをモデルの一部にします。

```python
inputs = keras.Input(shape=input_shape)
x = preprocessing_layer(inputs)
outputs = rest_of_the_model(x)
model = keras.Model(inputs, outputs)
```

このオプションを使用すると、モデルの残りの実行と同期してデバイス上で前処理が行われるため、GPU アクセラレーションの恩恵を受けることができます。GPU でトレーニングしている場合、これは`Normalization`レイヤー、およびすべての画像前処理レイヤーとデータ増強レイヤーに最適なオプションです。

**オプション 2:** これを`tf.data.Dataset`に適用して、次のように前処理されたデータのバッチを生成するデータセットを取得します。

```python
dataset = dataset.map(
  lambda x, y: (preprocessing_layer(x), y))
```

このオプションを使用すると、前処理は CPU で非同期に行われ、モデルに入れる前にバッファリングされます。

これは、`TextVectorization`およびすべての構造化データ前処理レイヤーに最適なオプションです。CPU でトレーニングしていて、画像前処理レイヤーを使用している場合にも、これは良いオプションです。

## 推論時にモデル内で前処理を行うことの利点

オプション 2 を選択した場合でも、後で前処理レイヤーを含む推論のみのエンドツーエンドモデルをエクスポートしたい場合があります。これを行う主な利点は、**モデルを移植可能にする**ことと、**[トレーニング/サービングスキュー](https://developers.google.com/machine-learning/guides/rules-of-ml#training-serving_skew)**を減らせることです。

すべてのデータ前処理がモデルの一部である場合、他のユーザーは、各特徴がどのようにエンコードおよび正規化されるかを知らなくても、モデルを読み込んで使用できます。推論モデルは生の画像または生の構造化データを処理できるようになり、モデルのユーザーは画像のピクセル値が`[-1, +1]`、または、`[0, 1]`に正規化されていても、テキストに使用されるトークン化スキーム、カテゴリカルフィーチャに使用されるインデックススキームの詳細を認識する必要はありません。これは、モデルを TensorFlow.js などの別のランタイムにエクスポートする場合に特に有用です。JavaScript で前処理パイプラインを再実装する必要はありません。

最初に前処理レイヤーを`tf.data`パイプラインに配置した場合、前処理をパッケージ化する推論モデルをエクスポートできます。前処理レイヤーとトレーニングモデルをチェーンする新しいモデルをインスタンス化するだけです。

```python
inputs = keras.Input(shape=input_shape)
x = preprocessing_layer(inputs)
outputs = training_model(x)
inference_model = keras.Model(inputs, outputs)
```

## クイックレシピ

### 画像データ増強（デバイス上）

画像データ増強レイヤーはトレーニング中にのみアクティブになることに注意してください（`Dropout`レイヤーと同様）。

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# Create a data augmentation stage with horizontal flipping, rotations, zooms
data_augmentation = keras.Sequential(
    [
        preprocessing.RandomFlip("horizontal"),
        preprocessing.RandomRotation(0.1),
        preprocessing.RandomZoom(0.1),
    ]
)

# Create a model that includes the augmentation stage
input_shape = (32, 32, 3)
classes = 10
inputs = keras.Input(shape=input_shape)
# Augment images
x = data_augmentation(inputs)
# Rescale image values to [0, 1]
x = preprocessing.Rescaling(1.0 / 255)(x)
# Add the rest of the model
outputs = keras.applications.ResNet50(
    weights=None, input_shape=input_shape, classes=classes
)(x)
model = keras.Model(inputs, outputs)

[画像分類を最初から行う](https://keras.io/examples/vision/image_classification_from_scratch/)の例で同様の設定が実際に行われていることを確認できます。

### 数値的特徴の正規化

In [ ]:
# Load some data
(x_train, y_train), _ = keras.datasets.cifar10.load_data()
x_train = x_train.reshape((len(x_train), -1))
input_shape = x_train.shape[1:]
classes = 10

# Create a Normalization layer and set its internal state using the training data
normalizer = preprocessing.Normalization()
normalizer.adapt(x_train)

# Create a model that include the normalization layer
inputs = keras.Input(shape=input_shape)
x = normalizer(inputs)
outputs = layers.Dense(classes, activation="softmax")(x)
model = keras.Model(inputs, outputs)

# Train the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")
model.fit(x_train, y_train)

### ワンホットエンコーディングによる文字列カテゴリカルフィーチャのエンコーディング

In [ ]:
# Define some toy data
data = tf.constant([["a"], ["b"], ["c"], ["b"], ["c"], ["a"]])

# Use StringLookup to build an index of the feature values and encode output.
lookup = preprocessing.StringLookup(output_mode="binary")
lookup.adapt(data)

# Convert new test data (which includes unknown feature values)
test_data = tf.constant([["a"], ["b"], ["c"], ["d"], ["e"], [""]])
encoded_data = lookup(test_data)
print(encoded_data)

インデックス 0 は、欠落している値（空の文字列 `""` として指定）のために予約されていることに注意してください。インデックス 1 は、語彙外の値（`adapt()` の際に表示されなかった値）のために予約されています。これは、`StringLookup` の `mask_token` および `oov_token` コンストラクタ引数を使用して構成できます。

`StringLookup` が実際に使用されている様子を[ゼロからの構造化データの分類](https://keras.io/examples/structured_data/structured_data_classification_from_scratch/)の例でご覧ください。

### ワンホットエンコーディングによる整数カテゴリカルフィーチャのエンコーディング

In [ ]:
# Define some toy data
data = tf.constant([[10], [20], [20], [10], [30], [0]])

# Use IntegerLookup to build an index of the feature values and encode output.
lookup = preprocessing.IntegerLookup(output_mode="binary")
lookup.adapt(data)

# Convert new test data (which includes unknown feature values)
test_data = tf.constant([[10], [10], [20], [50], [60], [0]])
encoded_data = lookup(test_data)
print(encoded_data)

インデックス 0 は、欠落している値（0 として指定）のために予約されていることに注意してください。インデックス 1 は、語彙外の値（`adapt()` の際に表示されなかった値）のために予約されています。これは、`StringLookup` の `mask_token` および <code>oov_token</code> コンストラクタ引数を使用して構成できます。

`IntegerLookup` が実際に使用されている様子を。[ゼロからの構造化データの分類](https://keras.io/examples/structured_data/structured_data_classification_from_scratch/)の例でご覧ください。

### 整数のカテゴリカルフィーチャにハッシュトリックを適用する

多くの異なる値（10 の 3 乗以上の桁）をとることができるカテゴリカルフィーチャがあり、各値がデータに数回しか表示されない場合、特徴値にインデックスを付けてワンホットエンコードすることは非現実的で効果的ではありません。このような場合は、代わりに、固定サイズのベクトルに値をハッシュする「ハッシュトリック」を適用することをお勧めします。これにより、特徴スペースのサイズが管理しやすくなり、明示的なインデックス作成が不要になります。

In [ ]:
# Sample data: 10,000 random integers with values between 0 and 100,000
data = np.random.randint(0, 100000, size=(10000, 1))

# Use the Hashing layer to hash the values to the range [0, 64]
hasher = preprocessing.Hashing(num_bins=64, salt=1337)

# Use the CategoryEncoding layer to one-hot encode the hashed values
encoder = preprocessing.CategoryEncoding(num_tokens=64, output_mode="binary")
encoded_data = encoder(hasher(data))
print(encoded_data.shape)

### トークンインデックスのシーケンスとしてテキストをエンコードする

以下は、`Embedded`レイヤーに渡されるテキストを前処理する方法です。

In [ ]:
# Define some text data to adapt the layer
data = tf.constant(
    [
        "The Brain is wider than the Sky",
        "For put them side by side",
        "The one the other will contain",
        "With ease and You beside",
    ]
)
# Instantiate TextVectorization with "int" output_mode
text_vectorizer = preprocessing.TextVectorization(output_mode="int")
# Index the vocabulary via `adapt()`
text_vectorizer.adapt(data)

# You can retrieve the vocabulary we indexed via get_vocabulary()
vocab = text_vectorizer.get_vocabulary()
print("Vocabulary:", vocab)

# Create an Embedding + LSTM model
inputs = keras.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = layers.Embedding(input_dim=len(vocab), output_dim=64)(x)
outputs = layers.LSTM(1)(x)
model = keras.Model(inputs, outputs)

# Call the model on test data (which includes unknown tokens)
test_data = tf.constant(["The Brain is deeper than the sea"])
test_output = model(test_data)

<a>テキスト分類を最初から行う</a>の例では、`Embedded`モードと組み合わされて<code>TextVectorization</code>レイヤーが動作する方法を確認できます。

このようなモデルをトレーニングする場合、最高のパフォーマンスを得るには、入力パイプラインの一部として`TextVectorization`レイヤーを使用する必要があることに注意してください（上記のテキスト分類の例で示すように）。

### マルチホットエンコーディングを使用した ngram の密な行列としてのテキストのエンコーディング

これは、`Dense`レイヤーに渡されるテキストを前処理する方法です。

In [ ]:
# Define some text data to adapt the layer
data = tf.constant(
    [
        "The Brain is wider than the Sky",
        "For put them side by side",
        "The one the other will contain",
        "With ease and You beside",
    ]
)
# Instantiate TextVectorization with "binary" output_mode (multi-hot)
# and ngrams=2 (index all bigrams)
text_vectorizer = preprocessing.TextVectorization(output_mode="binary", ngrams=2)
# Index the bigrams via `adapt()`
text_vectorizer.adapt(data)

print(
    "Encoded text:\n",
    text_vectorizer(["The Brain is deeper than the sea"]).numpy(),
    "\n",
)

# Create a Dense model
inputs = keras.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

# Call the model on test data (which includes unknown tokens)
test_data = tf.constant(["The Brain is deeper than the sea"])
test_output = model(test_data)

print("Model output:", test_output)

### TF-IDF 重み付けを使用した ngramの 密な行列としてのテキストのエンコード

これは、テキストを`Dense`レイヤーに渡す前に前処理する別の方法です。

In [ ]:
# Define some text data to adapt the layer
data = tf.constant(
    [
        "The Brain is wider than the Sky",
        "For put them side by side",
        "The one the other will contain",
        "With ease and You beside",
    ]
)
# Instantiate TextVectorization with "tf-idf" output_mode
# (multi-hot with TF-IDF weighting) and ngrams=2 (index all bigrams)
text_vectorizer = preprocessing.TextVectorization(output_mode="tf-idf", ngrams=2)
# Index the bigrams and learn the TF-IDF weights via `adapt()`
text_vectorizer.adapt(data)

print(
    "Encoded text:\n",
    text_vectorizer(["The Brain is deeper than the sea"]).numpy(),
    "\n",
)

# Create a Dense model
inputs = keras.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

# Call the model on test data (which includes unknown tokens)
test_data = tf.constant(["The Brain is deeper than the sea"])
test_output = model(test_data)
print("Model output:", test_output)